# Simple RNN (Recurrent Neural Network)

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input

In [2]:
sequence = np.array([1, 4, 9, 16, 25, 36, 49])
timesteps = sequence.shape[0]  # Get the number of elements in the sequence
data = sequence.reshape(1, timesteps, 1)  # Reshape to (1, timesteps, 1)

In [3]:
model = Sequential()
model.add(Input(shape=(timesteps, 1)))
model.add(SimpleRNN(units=50, return_sequences=True))  # Adjust units as needed
model.add(Dense(1, activation="linear"))

# Compile the model
model.compile(loss="mse", optimizer="adam")

In [4]:
# Predict the next number for the last element in the sequence
prediction = model.predict(data[:, -1:, :])  # Take the last element (timestep)

# Print the predicted value
print(f"Predicted next number: {prediction[0][0][0]:.2f}")  # Format the output

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Predicted next number: 3.29
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Predicted next number: 3.29
